In [30]:
%reset -f
%matplotlib inline
import numpy as np
import math
import random
import matplotlib
matplotlib.use('tkagg')
import matplotlib.pyplot as plt
import tqdm


# first we set all of the rate constants
k1 = 10     # PA binding to membrane sites to form MPA
k2 = 40       # PA dissociating from membrane sites
k3 = 0.008      # MPA binding to filament end
k4 = 1.3        # MPA dissociating from MPAF complex
k5 = 20       # conformational change from MPAF to MPF
k6 = 5         # reverse conform. change from MPF to MPAF
k7 = 100       # filament dissociating from MPF
k8 = 5         # filament binding back to MP
k9 = 5       # dissociation of P from membrane site M

# initialize all the dynamical variables
M = 10      # free membrane sites (number) within range of filaments
PA = 100    # profilin-actin (uM)
MPA = 0     # membrane bound profilin-actin (number)
MPAF = 0    # membrane bound profilin-actin-filament (number)
MPF =  0    # membrane bound profilin-filament (number)
MP = 0      # membrane bound profilin (number) in range of filaments
F = 10      # free filaments

# calculate the initial occupancy of the membrane sites based on affinity-why are you initializing twice?
Kmd = k2/k1 # this is the Kd for PA binding to M
MPA = math.floor((M*PA)/(Kmd+PA)) #mathfloor rounds numbers down
M = M-MPA


Ptot = [0]  # array to hold the total amount of polymer
time = 0    # variable to hold the current time
t = [0]     # array to hold sampled time points

Mtot=[0]
time2=0
t2= [0]

tend = 100  # end timepoint

# initialize variables to count the number of additions and 
# subtractions from the polymerized actin pool
deltaP = 0
deltaPmax = 100
 
while time < tend:
        # compute the current rates and put them into a vector
        rates = [ k1*M*PA,
                  k2*MPA,
                  k3*MPA*F,
                  k4*MPAF,
                  k5*MPAF,
                  k6*MPF,
                  k7*MPF,
                  k8*MP*F,
                  k9*MP,                
                  ]
        if time==0:
            print(rates)

        # calculate the sum of all reaction rates
        rate_sum = sum(rates)
        if rate_sum == 0:
                break
        # normalize all the reaction rates
        rates[:] = [x / rate_sum for x in rates] # array slice syntax for every element in the array
        # stairstep the normalized reaction rates from 0 - 1
        for i in range(1,9):
            rates[i]=rates[i]+rates[i-1]
                      
        # based on the summed rates, calculate the time increment for this reaction        
        tau = np.random.exponential(scale=1/rate_sum)  #random draw from an exponential distribution
        # increment the running timer with the new time step
        time = time + tau
        
        if time>=tend:
            print(rates)
            
        # roll the dice to choose the reaction that will occur in this time step
        rand = random.uniform(0,1) #choose next event

        # run through all the choices
        if rand <= rates[0]:
                M = M - 1
                MPA= MPA + 1
                Mtot.append(Mtot[-1] + 1)
                #t.append(time)

        elif rand <= rates[1]:
                MPA = MPA - 1
                Mtot.append(Mtot[-1] - 1)
                #t.append(time)
                M = M + 1

        elif rand <= rates[2]:
                MPA = MPA- 1
                Mtot.append(Mtot[-1] - 1)
                #t.append(time)
                F = F - 1
                MPAF = MPAF + 1
                
        elif rand <= rates[3]:
                MPAF = MPAF - 1
                MPA = MPA + 1
                Mtot.append(Mtot[-1] + 1)
                #t.append(time)
                F = F + 1

        elif rand <= rates[4]:
                MPAF = MPAF - 1
                MPF = MPF + 1
                # if we add to the polymer increment Ptot and take snapshot
                Ptot.append(Ptot[-1] + 1)
                t.append(time)

        elif rand <= rates[5]:
                MPF = MPF - 1
                MPAF = MPAF + 1 
                # if we subtract from the polymer decrement Ptot and take snapshot
                Ptot.append(Ptot[-1] - 1)
                t.append(time)

        elif rand <= rates[6]:
                MPF = MPF -1
                MP = MP + 1
                F = F + 1

        elif rand <= rates[7]:
                MP = MP - 1
                MPF = MPF + 1
                F = F - 1

        else:
                MP = MP -1
                M = M + 1

#matplotlib notebook
fig=plt.figure()
ax=fig.add_axes([0,0,1,1])
ax.plot(t,Ptot, 'ro')
ax.set_title("total polymer")
ax.set_xlabel("time (sec)")
ax.set_label( "polymer")

#print(Mtot)
#fig=plt.figure()
#ax=fig.add_axes([0,0,1,1])
#ax.plot(t,Mtot,'co')
#ax.set_title("total membrane-p-a")
#ax.set_xlabel("time (sec)")
#ax.set_label( "MPA")
# Run the calculations under the progress bar wrapper. 
#from tqdm.notebook import trange, tqdm
#from time import sleep

#for i in range(3, desc='1st loop'):
 #   for j in tqdm(range(100), desc='2nd loop'):
  #      sleep(0.01)


[1000, 360, 0.7200000000000001, 0.0, 0, 0, 0, 0, 0]
[0.0, 0.998003992015968, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [32]:
print(M)

1


In [9]:
6*3

18